# GPU 확인

In [1]:
import warnings
# 특정 UserWarning을 무시
warnings.filterwarnings("ignore", category=UserWarning)
import torch

# GPU 사용 가능 -> True, GPU 사용 불가 -> False
print(torch.cuda.is_available())

print(torch.__version__)

# GPU 이름 체크(cuda:0에 연결된 그래픽 카드 기준)
print(torch.cuda.get_device_name(device = 0)) # 'NVIDIA TITAN X (Pascal)'

# 사용 가능 GPU 개수 체크
print(torch.cuda.device_count()) # 3

c:\Users\705-18\anaconda3\envs\detectron\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True
1.10.0+cu113
NVIDIA GeForce RTX 3060
1


# 라이브러리 import

In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt

# 모델 정의
keypoint_rcnn

In [3]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE='cuda' #만약 gpu를 사용한다면 ‘cuda’로 수정
predictor = DefaultPredictor(cfg)

# 사진 출력 함수

In [4]:
# Jupyter Notebook에서 이미지를 출력하는 함수
def show_image(frame, title="Image"):
    # BGR 이미지를 RGB로 변환 (OpenCV는 BGR 형식, matplotlib은 RGB 형식)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.title(title)
    plt.imshow(rgb_frame)
    plt.axis('off')  # 축 숨기기
    plt.show()

# 이어질 관절 정의

In [5]:
skeleton_connections = [
                (0, 1), (0, 2),        # 코에서 두 눈으로 연결
                (1, 3), (2, 4),        # 왼쪽/오른쪽 눈에서 귀로 연결
                (5, 6),                # 왼쪽 어깨와 오른쪽 어깨 연결
                (5, 7), (7, 9),        # 왼쪽 어깨에서 팔꿈치, 팔꿈치에서 손목으로 연결
                (6, 8), (8, 10),       # 오른쪽 어깨에서 팔꿈치, 팔꿈치에서 손목으로 연결
                (5, 11), (6, 12),      # 왼쪽 어깨에서 왼쪽 엉덩이, 오른쪽 어깨에서 오른쪽 엉덩이로 연결
                (11, 12),              # 왼쪽 엉덩이와 오른쪽 엉덩이 연결
                (11, 13), (13, 15),    # 왼쪽 엉덩이에서 무릎, 무릎에서 발목 연결
                (12, 14), (14, 16)     # 오른쪽 엉덩이에서 무릎, 무릎에서 발목 연결
            ]

# 각도 계산 함수

In [6]:
def calculate_angle(point1, point2, point3):
    """
        각도 계산 함수
    """
    # 방향 벡터 계산
    vector1 = np.array(point2) - np.array(point1)  # p1 -> p2
    vector2 = np.array(point3) - np.array(point2)  # p2 -> p3
    
    # 내적 계산
    dot_product = np.dot(vector1, vector2)
    
    # 크기 계산
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    # 코사인 각도 계산
    cos_angle = dot_product / (magnitude1 * magnitude2)
    
    # 각도 (라디안) 계산
    angle_rad = np.arccos(cos_angle)
    
    # 라디안에서 도로 변환
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

# min-max 스케일러

In [7]:
def min_max_scale(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

# 학습데이터 생성

In [16]:
import pandas as pd
df = pd.DataFrame(columns=['elbow_angle1', 'shoulder_angle1', 'hip_angle1', 'knee_angle1', 'head_y1', 'hip_y1', 'ankle_y1',
                           'elbow_angle2', 'shoulder_angle2', 'hip_angle2', 'knee_angle2', 'head_y2', 'hip_y2', 'ankle_y2',
                           'elbow_angle3', 'shoulder_angle3', 'hip_angle3', 'knee_angle3', 'head_y3', 'hip_y3', 'ankle_y3',
                           'elbow_angle4', 'shoulder_angle4', 'hip_angle4', 'knee_angle4', 'head_y4', 'hip_y4', 'ankle_y4',
                           'elbow_angle5', 'shoulder_angle5', 'hip_angle5', 'knee_angle5', 'head_y5', 'hip_y5', 'ankle_y5',
                           'label'])
df

,elbow_angle1,shoulder_angle1,hip_angle1,knee_angle1,head_y1,hip_y1,ankle_y1,elbow_angle2,shoulder_angle2,hip_angle2,...,hip_y4,ankle_y4,elbow_angle5,shoulder_angle5,hip_angle5,knee_angle5,head_y5,hip_y5,ankle_y5,label


0: 정상   
1: 쓰러짐   
2: 추락

In [ ]:
import copy

# 이미지들이 저장된 폴더 경로
image_folder = './b'

# 이미지 파일 목록을 불러오기
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')])

temp_que = []
i = 0
# 이미지 처리 루프
for image_file in image_files:
    # 이미지 경로 생성
    image_path = os.path.join(image_folder, image_file)
    # 이미지 읽어오기
    frame = cv2.imread(image_path)
    # 이미지 크기 가져오기
    image_height, image_width, channels = frame.shape
    
    if frame is None:
        print(f"이미지 로드에 실패했습니다: {image_file}")
        continue
    
    # Detectron2를 이용한 관절 검출
    try:
        keypoints = predictor(frame)['instances'].get_fields()['pred_keypoints'][0]
    except:
        continue
    # 관절 각도 계산
    # 팔꿈치 각도
    elbow_angle = calculate_angle((int(keypoints[6][0]), int(keypoints[6][1])), (int(keypoints[8][0]), int(keypoints[8][1])), (int(keypoints[10][0]), int(keypoints[10][1])))  
    # 어깨 각도 
    # 어깨 각도를 오른 팔꿈치-오른 어깨-왼쪽 어깨 에서 오른 팔꿈치-오른 어깨-오른 엉덩이로 바꿈
    shoulder_angle = calculate_angle((int(keypoints[6][0]), int(keypoints[6][1])), (int(keypoints[8][0]), int(keypoints[8][1])), (int(keypoints[12][0]), int(keypoints[12][1]))) 
    # 허리 각도 
    hip_angle = calculate_angle((int(keypoints[6][0]), int(keypoints[6][1])), (int(keypoints[12][0]), int(keypoints[12][1])), (int(keypoints[13][0]), int(keypoints[13][1])))  
    # 무릎 각도
    knee_angle = calculate_angle((int(keypoints[12][0]), int(keypoints[12][1])), (int(keypoints[14][0]), int(keypoints[14][1])), (int(keypoints[16][0]), int(keypoints[16][1])))  

    # 높이(y) 추출
    head_y = min_max_scale(int(keypoints[0][1]), 0, image_height)
    hip_y = min_max_scale(int(keypoints[12][1]), 0, image_height)
    ankle_y = min_max_scale(int(keypoints[16][1]), 0, image_height)
    
    f_lst = [elbow_angle, shoulder_angle, hip_angle, knee_angle, head_y, hip_y, ankle_y]


    # 프레임 카운트
    if i < 5:
        i += 1
        for f in f_lst:
            temp_que.append(f)
    elif i <= 5:
        temp_que = temp_que[7:]
        for f in f_lst:
            temp_que.append(f)

        show_image(frame)

        label = ''
        while label == '':
            label = input()
            if len(label) >= 2:
                label = label[:1]
        label_que = copy.deepcopy(temp_que)
        label_que.append(label)
        temp_df = pd.DataFrame([label_que], columns=['elbow_angle1', 'shoulder_angle1', 'hip_angle1', 'knee_angle1', 'head_y1', 'hip_y1', 'ankle_y1',
                                                    'elbow_angle2', 'shoulder_angle2', 'hip_angle2', 'knee_angle2', 'head_y2', 'hip_y2', 'ankle_y2',
                                                    'elbow_angle3', 'shoulder_angle3', 'hip_angle3', 'knee_angle3', 'head_y3', 'hip_y3', 'ankle_y3',
                                                    'elbow_angle4', 'shoulder_angle4', 'hip_angle4', 'knee_angle4', 'head_y4', 'hip_y4', 'ankle_y4',
                                                    'elbow_angle5', 'shoulder_angle5', 'hip_angle5', 'knee_angle5', 'head_y5', 'hip_y5', 'ankle_y5',
                                                    'label'])
        df = pd.concat([df, temp_df], ignore_index=True)

In [19]:
df.dropna()

,elbow_angle1,shoulder_angle1,hip_angle1,knee_angle1,head_y1,hip_y1,ankle_y1,elbow_angle2,shoulder_angle2,hip_angle2,...,hip_y4,ankle_y4,elbow_angle5,shoulder_angle5,hip_angle5,knee_angle5,head_y5,hip_y5,ankle_y5,label
0,21.150131,101.943679,82.524193,10.947001,0.061111,0.291667,0.579630,24.471777,114.309917,85.853859,...,0.286111,0.613889,66.877651,81.158185,19.150421,20.002210,0.095370,0.295370,0.599074,0
1,24.471777,114.309917,85.853859,38.616091,0.091667,0.307407,0.589815,77.640569,89.197077,47.126965,...,0.295370,0.599074,56.918360,81.943344,34.459092,34.505583,0.109259,0.305556,0.591667,0
2,77.640569,89.197077,47.126965,16.860770,0.071296,0.289815,0.595370,78.910887,83.603691,40.907312,...,0.305556,0.591667,41.375595,97.142929,19.382454,20.417719,0.125000,0.319444,0.617593,0
3,78.910887,83.603691,40.907312,20.423130,0.075926,0.286111,0.613889,66.877651,81.158185,19.150421,...,0.319444,0.617593,36.573031,92.064508,30.902033,133.113000,0.135185,0.340741,0.446296,0
4,66.877651,81.158185,19.150421,20.002210,0.095370,0.295370,0.599074,56.918360,81.943344,34.459092,...,0.340741,0.446296,51.215971,79.597420,29.020237,82.192047,0.149074,0.364815,0.539815,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,78.594839,127.968720,47.155779,148.508716,0.710185,0.832407,0.838889,27.925395,81.268200,41.527131,...,0.818519,0.854630,18.307057,88.235699,23.118983,134.860292,0.698148,0.793519,0.837963,2
66,27.925395,81.268200,41.527131,145.647652,0.718519,0.828704,0.849074,17.622961,95.519310,35.117220,...,0.793519,0.837963,20.999781,88.252466,22.860288,136.587911,0.698148,0.789815,0.837963,2
67,17.622961,95.519310,35.117220,139.995163,0.712963,0.820370,0.850000,11.879917,105.734282,35.566930,...,0.789815,0.837963,100.051781,3.079588,31.743125,142.454547,0.703704,0.804630,0.834259,2
68,11.879917,105.734282,35.566930,142.511161,0.728704,0.818519,0.854630,18.307057,88.235699,23.118983,...,0.804630,0.834259,136.424974,44.978148,79.584162,138.182315,0.728704,0.797222,0.830556,2


In [26]:
result_df.to_csv('안전사고 분류 가중치2.csv', index=None)

# 분류 테스트

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

df = pd.read_excel('t.xlsx')

X = df.drop(columns=['label'])  # 특징 (feature)
y = df['label']   

# 훈련 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [16]:
import pandas as pd

temp_df = pd.read_excel('t.xlsx')

In [17]:
df = pd.concat([df, temp_df], ignore_index=True)

In [18]:
len(df)

340

In [20]:
df.to_csv('안전사고 분류 학습 데이터.csv')